<img src="../../img/change_point_detection.png" alt="change point detection" style="width: 100%; border-radius: 20px;"/>

## Objective

## Scope

In [2]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from skmultiflow.drift_detection import DDM
from skmultiflow.drift_detection.adwin import ADWIN
import plotly.express as px
import plotly.graph_objects as go

from utils.drift import *

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 0️⃣ Specify your paths </p>

In order to run the notebook, the following file is required:
- Time series of bird sightings: *[time_series_27_species.csv](https://drive.google.com/drive/folders/18XoTsDtWnN4QdIBNGGbq-jaa6U3nVb2e)*

In [3]:
path_sightings = '../../../01_Data/datasets/time_series_27_species.csv'

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 1️⃣ Load data </p>

We use the pygmy owl as example.

In [4]:
df = pd.read_csv(path_sightings)
pygmy_owl = df[(df.name_species == 'Sperlingskauz') & (df.eea_grid_id == '50kmE4100N2700')].reset_index(drop=True)
df

,name_species,eea_grid_id,date,n_sightings
0,Haubentaucher,50kmE4200N3300,2018-01-01,12.0
1,Haubentaucher,50kmE4200N3300,2018-01-02,11.0
2,Haubentaucher,50kmE4200N3300,2018-01-03,0.0
3,Haubentaucher,50kmE4200N3300,2018-01-04,7.0
4,Haubentaucher,50kmE4200N3300,2018-01-05,7.0
...,...,...,...,...
11339455,Gänsegeier,50kmE4150N2500,2022-12-27,0.0
11339456,Gänsegeier,50kmE4150N2500,2022-12-28,0.0
11339457,Gänsegeier,50kmE4150N2500,2022-12-29,0.0
11339458,Gänsegeier,50kmE4150N2500,2022-12-30,0.0


In [5]:
gelbspoetter = df[(df.name_species == 'Gelbspötter') & (df.eea_grid_id == '50kmE4350N3250')].reset_index(drop=True)

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣ Change point detection </p>


In [6]:
sighting_stream = pygmy_owl['n_sightings'].values

## 1. ADWIN

In [7]:
import random
from river import drift

adwin = drift.ADWIN()
change_points_adwin_river = []
for i, val in enumerate(sighting_stream):
    adwin.update(val)
    if adwin.drift_detected:
        change_points_adwin_river.append(i)

plot_change_points(data=pygmy_owl, 
                   change_points=change_points_adwin_river, 
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700', 
                   legend_title='detected change point (ADWIN)')

In [8]:
import random
from river import drift

kswin = drift.KSWIN(seed=42, window_size=100)
change_points_kswin = []
for i, val in enumerate(sighting_stream):
    kswin.update(val)
    if kswin.drift_detected:
        change_points_kswin.append(i)

plot_change_points(data=pygmy_owl, 
                   change_points=change_points_kswin, 
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700', 
                   legend_title='detected change point (KSWIN)')

In [9]:
ph = drift.PageHinkley()
change_points_ph = []
for i, val in enumerate(sighting_stream):
    ph.update(val)
    if ph.drift_detected:
        change_points_ph.append(i)

plot_change_points(data=pygmy_owl, 
                   change_points=change_points_ph, 
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700', 
                   legend_title='detected change point (PageHinkley)')

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣ Smoothing </p>


In [10]:
smoothed_pygmy_owl = pygmy_owl.copy()
smoothed_pygmy_owl['n_sightings'] = pygmy_owl['n_sightings'].rolling(window=30, center=True).mean()

In [11]:
smoothed_pygmy_owl = pygmy_owl.copy()
smoothed_pygmy_owl['n_sightings'] = pygmy_owl['n_sightings'].rolling(window=30, center=True).mean()
smoothed_sighting_stream = smoothed_pygmy_owl['n_sightings'].values

adwin = drift.KSWIN(window_size=300)
change_points_adwin_river = []
for i, val in enumerate(smoothed_sighting_stream):
    adwin.update(val)
    if adwin.drift_detected:
        change_points_adwin_river.append(i)

plot_change_points(data=smoothed_pygmy_owl, 
                   change_points=change_points_adwin_river, 
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700', 
                   legend_title='detected change point (KSWIN)')

In [12]:
smoothed_gelbspoetter = gelbspoetter.copy()
smoothed_gelbspoetter['n_sightings'] = smoothed_gelbspoetter['n_sightings'].rolling(window=30, center=True).mean()
smoothed_stream = smoothed_gelbspoetter['n_sightings'].values

adwin = drift.KSWIN(window_size=100)
change_points_adwin_river = []
for i, val in enumerate(smoothed_stream):
    adwin.update(val)
    if adwin.drift_detected:
        change_points_adwin_river.append(i)

plot_change_points(data=smoothed_gelbspoetter, 
                   change_points=change_points_adwin_river, 
                   title='Yellow mocker (Gelbspötter) sightings in grid 50kmE4350N3250', 
                   legend_title='detected change point (KSWIN)')

## Migration Drift

In [13]:
plot_change_points_per_year(data=smoothed_gelbspoetter, 
                   change_points=change_points_adwin_river, 
                   title='Yellow mocker (Gelbspötter) sightings in grid 50kmE4350N3250', 
                   legend_title='detected change point (KSWIN)',
                   show_first_sighting=False)

In [14]:
plot_change_points_per_year(data=smoothed_gelbspoetter, 
                   change_points=change_points_adwin_river, 
                   title='Yellow mocker (Gelbspötter) sightings in grid 50kmE4350N3250', 
                   legend_title='detected change point (KSWIN)',
                   show_first_sighting=True)

## Habitatsdrift

In [15]:
smoothed_pygmy_owl = pygmy_owl.copy()
smoothed_pygmy_owl['n_sightings'] = pygmy_owl['n_sightings'].rolling(window=30, center=True).mean()
smoothed_sighting_stream = smoothed_pygmy_owl['n_sightings'].values

adwin = drift.KSWIN(window_size=300)
change_points_adwin_river = []
for i, val in enumerate(smoothed_sighting_stream):
    adwin.update(val)
    if adwin.drift_detected:
        change_points_adwin_river.append(i)

plot_change_points_per_year(data=smoothed_pygmy_owl, 
                   change_points=change_points_adwin_river, 
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700', 
                   legend_title='detected change point (KSWIN)')